In [1]:
import os
import yaml
import json
import pandas as pd

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import wombat.bsub as bsub

load sample data

In [5]:
# m = yaml.safe_load(open('../tests/data/cnv/C3L-00677.yaml'))
# m

In [6]:
# m['wxs']['normal']['filepath'] = '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/'

make test input yaml

In [7]:
# template = yaml.safe_load(open('/home/estorrs/pecgs-pipeline/submodules/pecgs-cnv/cwl/template.cnv_workflow.yaml'))
# template

In [8]:
# template['common_biallelic']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf'
# template['reference_dir']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1'
# template['pool_of_normals']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/create_pon/gatk4scnaPON.Normal.hdf5'
# template['target_interval_list']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.autosome_only.bed.preprocessed.exome.interval_list'
# template['protein_coding_gene']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/gencode.v34.annotation.gene_filtered.protein_coding.ensembl_ID_no_version.protein-coding_hgnc_filtered.duplicates_removed.ensembl_ID_removed.txt'
# template['cytoband']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/cytoBand.txt'

# template['normal_bam']['path'] = m['wxs']['normal']['filepath']
# template['tumor_bam']['path'] = m['wxs']['tumor']['filepath']
# template['sample'] = m['id']




In [9]:
# template

In [10]:
# yaml.safe_dump(template, open('../tests/data/cnv/C3L-00677_inputs.yaml', 'w'))

In [11]:
template = yaml.safe_load(open('../tests/data/cnv/C3L-00677_inputs.yaml'))
template

{'common_biallelic': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf'},
 'cytoband': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/cytoBand.txt'},
 'normal_bam': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'},
 'pool_of_normals': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/create_pon/gatk4scnaPON.Normal.hdf5'},
 'protein_coding_gene': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/gencode.v34.annotation.gene_filtered.protein_coding.ensembl_ID_no_version.protein-coding_hgnc_filtered.duplicates_removed.ensembl_ID_removed.txt'},
 'reference_dir': {'class': 'Directory',
  'path': '/storag

make cromwell compute1 config

In [12]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/cnv/C3L-00677'
output_fp = '../tests/data/cnv/C3L-00677.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

make execution script

In [13]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/cnv'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'C3L-00677.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'submodules', 'pecgs-cnv', 'cwl', 'cnv.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'C3L-00677_inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab', '/storage1/fs1/m.wyczalkowski', '/scratch1/fs1/dinglab']

In [14]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'gpu_model': 'TeslaV100_SXM2_32GB',
 'gpu_mem': '30',
 'gpu_num': 1,
 'use_gpu': False,
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [15]:
run_commands = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes, workflow_root=workflow_root)
filepath = os.path.join('../tests/data/cnv/run_job.sh')
bsub.write_command_file(run_commands, filepath)

In [16]:
for c in run_commands: print(c)

mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/cnv/C3L-00677/cromwell-workdir/logs
source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/cnv:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/cnv /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/cnv/C3L-00677:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/cnv/C3L-00677 /storage1/fs1/dinglab:/storage1/fs1/dinglab /storage1/fs1/m.wyczalkowski:/storage1/fs1/m.wyczalkowski /scratch1/fs1/dinglab:/scratch1/fs1/dinglab"
bgadd -L 10 /estorrs/test_cromwell
export PATH="/opt/java/openjdk/bin:$PATH"
bsub -R 'select[mem>10GB] rusage[mem=10GB] span[hosts=1]' -M 11GB -n 1 -q dinglab -G compute-dinglab -a 'docker(estorrs/cromwell-runner:58)' -g /estorrs/test_cromwell -J 